In [1]:
# Run below commands in google colab
# install Java8
!apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 

In [2]:
# unzip the spark file
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# install findspark 
!pip install -q findspark

In [3]:
# set environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = ( SparkSession.builder
                      .master("local[*]")
                      #.config("spark.sql.warehouse.dir", "./")
                      #.config("spark.some.config.option", "some-value")
                      .getOrCreate() 
                      )

sc = spark.sparkContext #spark context

In [7]:
health= ( spark.read
                              .load("/content/healthcare-dataset-stroke-data.csv",
                                    format = 'csv',
                                    sep=",", 
                                    #schema = schema, #you can specify schema explicitly
                                    inferSchema="true", #you can try to infer schema
                                    header="true")
)

#check the first rows
health.show()#health dataset

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|null|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [8]:
health.groupby('work_type','stroke').count().show()#count the no of strokes in a department

+-------------+------+-----+
|    work_type|stroke|count|
+-------------+------+-----+
| Never_worked|     0|   22|
|      Private|     0| 2776|
|Self-employed|     0|  754|
|     Govt_job|     1|   33|
|      Private|     1|  149|
|     children|     1|    2|
|     Govt_job|     0|  624|
|Self-employed|     1|   65|
|     children|     0|  685|
+-------------+------+-----+



insight: private worktype has more no of strokes.

In [9]:
health.groupby('gender','stroke').count().show()#count of no of female/male having stroke

+------+------+-----+
|gender|stroke|count|
+------+------+-----+
|  Male|     1|  108|
| Other|     0|    1|
|Female|     0| 2853|
|  Male|     0| 2007|
|Female|     1|  141|
+------+------+-----+



insight:0, there are no strokes in female/male gender

In [10]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
health.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in health.columns]
   ).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|201|             0|     0|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+



In [ ]:
from pyspark.sql.functions import regexp_replace#replaacing null valves
health1=health.withColumn('smoking_status', regexp_replace('smoking_status', 'Unknown', 'No info'))
health1.show()   

In [ ]:
from pyspark.sql.functions import mean
mean = health1.select(mean(health['bmi'])).collect()
mean_bmi = mean[0][0]
health2= health1.na.fill(mean_bmi,['bmi'])

In [ ]:
import pyspark.sql.functions as f
Health_new = health2.select('*').where((health2.age>60) &
                         (health2.age<70))
Health_new.show()

In [ ]:
Health_new.write.json('Users\dilee')

In [ ]:
!pip install tensorflow-io
!pip install kafka-python

In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

In [ ]:
pip install kafka-python

In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/2.8.2/kafka_2.13-2.8.2.tgz
!tar -xvf kafka_2.13-2.8.2.tgz

In [ ]:
!./kafka_2.13-2.8.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.8.2/config/zookeeper.properties


In [ ]:
!./kafka_2.13-2.8.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-2.8.2/config/server.properties


In [ ]:
!ps -ef | grep kafka

In [ ]:
!./kafka_2.13-2.8.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --topic dileep10 --replication-factor 1


In [ ]:
!./kafka_2.13-2.8.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.8.2/config/zookeeper.properties

In [ ]:
!./kafka_2.13-2.8.2/bin/kafka-console-producer.sh --bootstrap-server 127.0.0.1:9092 --topic dileep10 < health_sample.json

In [ ]:
!./kafka_2.13-2.8.2/bin/kafka-console-consumer.sh --topic dileep10--from-beginning --bootstrap-server 127.0.0.1:9092 

In [ ]:
from kafka import KafkaProducer
import json 
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],value_serializer=lambda v: json.dumps(v).encode('utf-8'))
with open("/content/health_sample.json", 'r') as file:
    read = file.read()
    for i in range(1):
        producer.send("dileep1", value=read)
producer.close()

In [ ]:
from kafka import KafkaConsumer
import sys
bootstrap_servers = ['127.0.0.1:9092']
topicName = 'dileep1'
consumer = KafkaConsumer (topicName, group_id = 'group1',bootstrap_servers = bootstrap_servers,
auto_offset_reset = 'earliest')

In [ ]:
try:
    for message in consumer:
        print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,message.offset, message.key,message.value))
except KeyboardInterrupt:
    sys.exit()

In [ ]:
!./kafka_2.13-2.8.2/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --from-beginning --topic dileep1 > dileep.csv

In [ ]:
!./kafka_2.13-2.8.2/bin/connect-standalone.sh ./kafka_2.13-2.8.2/config/connect-standalone.properties ./kafka_2.13-2.8.2/config/connect-console-sink.properties

In [ ]:
pip install mysql

In [ ]:
pip install mysql.connector

In [ ]:
pip install mysql

In [ ]:
# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/

In [ ]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root',  
                        password='root')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE dileep2")
        print("Database is created")
        
except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
from kafka import KafkaConsumer

In [ ]:
from mysql.connector import cursor
import mysql
Topic='dileep10'
conn=mysql.connector.connect(user='root',password='root',host='localhost',port="3306",database='dileep2')
conn.commit()
cursor=conn.cursor()
sql="""insert into details(id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
print('connecting to kafka')
consumer=KafkaConsumer(Topic)
print("connected to kafka")
print(f"reading messsages from {Topic}")
for i,row in consumer:
  message=msg.value.decode("utf-8")
  (id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke)=message.split(',')
  val=(id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke)
  cursor.excute(sql,val)
  print(f"{id},{gender},{age},{hypertension},{heart_disease},{ever_married},{work_type},{residence_type},{avg_glucose_level},{bmi},{smoking_status},{stroke}")
  print(f"A{age} was inserted into database")
  conn.commit()
conn.close()

In [ ]:
Health_new.show()

In [ ]:
pandasDF = Health_new.toPandas()
print(pandasDF)
d2=pandasDF[:5]
d2

In [ ]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', database='dileep2', user='root', password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS details;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE details(id int(23), gender varchar(255),age int(255),hypertension int(25),heart_disease int(25),ever_married varchar(255),work_type varchar(255),Residence_type varchar(255),avg_glucose_level int(255),bmi int(255),smoking_status varchar(255),stroke int(23))")
        print("Table is created....")
        #loop through the data frame
        for i,row in d2.iterrows():
            #here %S means string values 
            sql = "INSERT INTO dileep2.details VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)